# model TPH

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split 
# Import Halving Grid Search
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from scipy.stats import kurtosis,skew
from numpy import mean,sqrt,square
from scipy.fftpack import fft, fftfreq
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
# Función que filtra tags por alta correlación
def corrkill(dataframe, features, corr_cut=0.8):

    df = dataframe[features]
    dfcorr = pd.DataFrame(np.triu(df.corr()), columns=df.columns, index=df.columns)
    dfcorr = dfcorr.stack().reset_index()
    dfcorr.columns = ['Feat1','Feat2','Val']
    dfcorr = dfcorr[~dfcorr['Val'].isin([0,1])]
    dfcorr = dfcorr[dfcorr['Val'].abs()>corr_cut]
    dfcorr["Val"]=dfcorr["Val"].abs()
    
    return list(dfcorr['Feat2']), dfcorr.sort_values(by="Val",ascending=False)

In [3]:
variables_interes=['pyrite law', 'chalcopyrite law',
       'chalcocite law', 'covelin law', 'crusher index', 'sag power index',
       'ball work index', 'bornite law','charge cell', 'TPH','water', 'speed','HH TPH', 'granulometry', 'Edad','loss of TPH',"power","solid percentage","HH charge cell","LL charge cell"]
variables_interes

['pyrite law',
 'chalcopyrite law',
 'chalcocite law',
 'covelin law',
 'crusher index',
 'sag power index',
 'ball work index',
 'bornite law',
 'charge cell',
 'TPH',
 'water',
 'speed',
 'HH TPH',
 'granulometry',
 'Edad',
 'loss of TPH',
 'power',
 'solid percentage',
 'HH charge cell',
 'LL charge cell']

In [ ]:
# read df full csv
df_full=pd.read_csv('../../data/consolidated data/df_full_01November2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')[variables_interes]#["2020":"2021"]
df_full

***Custom features***

In [ ]:
df_full["delta HH charge cell"]=df_full["HH charge cell"]-df_full["charge cell"]

In [ ]:
df_full["delta LL charge cell"]=df_full["charge cell"]-df_full["LL charge cell"]

In [ ]:
df_corr = df_full.dropna().corr()
corr_tph=pd.DataFrame(df_corr.abs().TPH.sort_values(ascending=False))
top_corr=corr_tph[corr_tph.TPH>=0]
top_corr

# Feature Engineering

In [ ]:
def feature_engineering(df):
      
    targets = ["TPH","Edad","loss of TPH","HH TPH"]
    df_vars = df.copy()
    column_inicial=df_vars.columns

    for column in df_vars:
    
        if (column not in targets):

            ## 10 min
            #df_vars[f"skew_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:skew(x))
            #df_vars[f"kurt_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:kurtosis(x))
            df_vars[f"mean_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:np.nanmean(list(x)))
            df_vars[f"max_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:np.nanmax(list(x)))
            #df_vars[f"sum_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:np.nansum(list(x)))
            df_vars[f"min_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:np.nanmin(list(x)))
            df_vars[f"rms_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x: sqrt(mean(square(list(x)))))
            df_vars[f"var_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x: np.nanvar(x))

            # 5 min
            df_vars[f"mean_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x:np.nanmean(list(x)))
            df_vars[f"max_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x:np.nanmax(list(x)))
            #df_vars[f"sum_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x:np.nansum(list(x)))
            df_vars[f"min_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x:np.nanmin(list(x)))
            df_vars[f"rms_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x: sqrt(mean(square(list(x)))))
            df_vars[f"var_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x: np.nanvar(x))

            # 3 min
            #df_vars[f"skew_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:skew(x))
            #df_vars[f"kurt_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:kurtosis(x))
            df_vars[f"mean_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:np.nanmean(list(x)))
            df_vars[f"max_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:np.nanmax(list(x)))
            #df_vars[f"sum_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:np.nansum(list(x)))
            df_vars[f"min_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:np.nanmin(list(x)))
            df_vars[f"rms_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x: sqrt(mean(square(list(x)))))
            df_vars[f"var_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x: np.nanvar(x))

            # Fourier
            #L=10
            #df_vars[f"fourier_{L}_max_{column}"]=df_vars[column].rolling(window=L).apply(lambda x:    np.nanmax(abs(fft(list(x))[1:int(L/2)]/(L/2))))
            #df_vars[f"fourier_{L}_min_{column}"]=df_vars[column].rolling(window=L).apply(lambda x:    np.nanmin(abs(fft(list(x))[1:int(L/2)]/(L/2))))
            #df_vars[f"fourier_{L}_mean_{column}"]=df_vars[column].rolling(window=L).apply(lambda x:    np.nanmean(abs(fft(list(x))[1:int(L/2)]/(L/2))))
            #df_vars[f"fourier_{L}_sum_{column}"]=df_vars[column].rolling(window=L).apply(lambda x:    np.nansum(abs(fft(list(x))[1:int(L/2)]/(L/2))))
#
            #L=5
            #df_vars[f"fourier_{L}_max_{column}"]=df_vars[column].rolling(window=L).apply(lambda x:    np.nanmax(abs(fft(list(x))[1:int(L/2)]/(L/2))))
            #df_vars[f"fourier_{L}_min_{column}"]=df_vars[column].rolling(window=L).apply(lambda x:    np.nanmin(abs(fft(list(x))[1:int(L/2)]/(L/2))))
            #df_vars[f"fourier_{L}_mean_{column}"]=df_vars[column].rolling(window=L).apply(lambda x:    np.nanmean(abs(fft(list(x))[1:int(L/2)]/(L/2))))
            #df_vars[f"fourier_{L}_sum_{column}"]=df_vars[column].rolling(window=L).apply(lambda x:    np.nansum(abs(fft(list(x))[1:int(L/2)]/(L/2))))


    for column in column_inicial:
        if (column not in targets):
            # lags  
            df_vars[f"{column}_(t-1)"] = df_vars[column].shift(1)
            df_vars[f'{column}_(t-2)'] = df_vars[column].shift(2)
            df_vars[f'{column}_(t-3)'] = df_vars[column].shift(3)
            df_vars[f'{column}_(t-4)'] = df_vars[column].shift(4)
            df_vars[f'{column}_(t-5)'] = df_vars[column].shift(5)
           
    
    return df_vars
   

In [ ]:
df_full.head()

In [ ]:
df_features=feature_engineering(df_full)

In [ ]:
# Se guarda en csv
df_features.to_csv('../../data/consolidated data/df_features_'+pd.to_datetime('today').strftime('%d%B%Y')+'.csv', index=True)

In [ ]:
df_features.shape

In [4]:
# read df full csv
df_features=pd.read_csv('../../data/consolidated data/df_features_25November2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')#["2020":"2021"]
df_features["TPH"]=df_features["TPH"].shift(-10)
df_features["loss of TPH"]=df_features["loss of TPH"].shift(-10)
df_features.dropna(inplace=True)
df_features.sort_index(inplace=True)
df_features.shape

(856091, 382)

In [5]:
# Verificando si hay duplicados
u=[i for i in df_features.columns if (not re.match(".*water",i)) and (not re.match(".*power",i)) ]
y=[i for i in df_features.columns if  re.match(".*sag power index",i)] 
u=y+u
df_features=df_features[u]
df_features.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

# Correlación entre variables

In [ ]:
# Se obtienen tags altamente correlacionados y matriz que permite visualizar cuales son los par de tags altamente correlacionados
collinearity, table_cor=corrkill(df_features,df_features.columns)

In [ ]:
# Tags altamente correlacionados
table_cor

In [ ]:
df_corr = df_features.dropna().corr()
corr_tph=pd.DataFrame(df_corr.abs().TPH.sort_values(ascending=False))


In [ ]:
top_corr=corr_tph[corr_tph.TPH>=0.2]
top_corr

# Equilibrio de eventos de perdidas de TPH

In [6]:
# Entrenamiento
entrenamiento=pd.concat([df_features["2020-01":"2020-03"],df_features["2020-05"],df_features["2020-07":"2020-12"],df_features["2021-01":"2021-08"],df_features["2021-10"],df_features["2021-12"]])
entrenamiento.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

In [7]:
# Validación
validacion=pd.concat([df_features["2020-06"],df_features["2021-11"]])
validacion.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

In [8]:
# Test
test=pd.concat([df_features["2020-04"],df_features["2021-09"],df_features["2022"]])
test.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

In [9]:
tags=df_features.columns.to_list()
tags.remove("loss of TPH")
#tags.remove("TPH")

In [10]:
# Sobremuestreo: Aumentar numero de datos de la clase minoritaria
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

# ROS:
Xtrain, Ytrain=ros.fit_resample(entrenamiento[tags],entrenamiento["loss of TPH"])

In [11]:
# Ytrain reg
#Ytrain=Xtrain["loss of TPH"]
Xtrain=Xtrain.drop(columns=["TPH"])

In [12]:
# Yval reg
Yval=validacion["loss of TPH"]
Xval=validacion.drop(columns=["loss of TPH","TPH"])

In [13]:
# Ytest reg
Ytest=test["loss of TPH"]
Xtest=test.drop(columns=["loss of TPH","TPH"])

In [14]:
#Unión del conjunto de Entrenamiento y Validación:
Entre_Vali_features=pd.concat([Xval,Xtrain])
Entre_Vali_target=pd.concat([Yval,Ytrain])

#Indices para separar ambos conjuntos:
Entre_indice=np.full(len(validacion),-1)
Vali_indice=np.full(len(entrenamiento),0)
Indices=np.append(Entre_indice,Vali_indice)
x=PredefinedSplit(Indices)

In [15]:
Color_loss=test["loss of TPH"]
Color_loss=Color_loss.astype(str)

# Feature selection

In [16]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        Xtrain.columns.to_list()
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBClassifier(seed=1,  eval_metric='mlogloss'))
    ]
)

In [17]:
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['sag power index',
                                                   'mean_sag power index_10',
                                                   'max_sag power index_10',
                                                   'min_sag power index_10',
                                                   'rms_sag power index_10',
                                                   'var_sag power index_10',
                                                   'mean_sag power index_5',
                                                   'max_sag power index_5',
                                                   'min_sag power index_5',
                                                   'rms_sag power index_5',
                                                   'var_sag power index_5',
                                                  

**General**

In [18]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest)
print(classification_report(Ytest,Ypred))

              precision    recall  f1-score   support

         0.0       0.92      0.80      0.86     94376
         1.0       0.64      0.84      0.73     39276

    accuracy                           0.81    133652
   macro avg       0.78      0.82      0.79    133652
weighted avg       0.84      0.81      0.82    133652



**2020**

In [20]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest["2020"])
print(classification_report(Ytest["2020"],Ypred))

              precision    recall  f1-score   support

         0.0       0.91      0.64      0.75     19501
         1.0       0.70      0.93      0.80     17936

    accuracy                           0.78     37437
   macro avg       0.81      0.79      0.78     37437
weighted avg       0.81      0.78      0.78     37437



**2021**

In [21]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest["2021"])
print(classification_report(Ytest["2021"],Ypred))

              precision    recall  f1-score   support

         0.0       0.95      0.91      0.93     28910
         1.0       0.71      0.81      0.76      7691

    accuracy                           0.89     36601
   macro avg       0.83      0.86      0.84     36601
weighted avg       0.90      0.89      0.89     36601



**2022**

In [22]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest["2022-01"])
print(classification_report(Ytest["2022-01"],Ypred))

              precision    recall  f1-score   support

         0.0       0.91      0.82      0.86     27412
         1.0       0.50      0.70      0.59      7086

    accuracy                           0.80     34498
   macro avg       0.71      0.76      0.73     34498
weighted avg       0.83      0.80      0.81     34498



In [23]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest["2022-02"])
print(classification_report(Ytest["2022-02"],Ypred))

              precision    recall  f1-score   support

         0.0       0.91      0.78      0.84     18553
         1.0       0.55      0.79      0.65      6563

    accuracy                           0.78     25116
   macro avg       0.73      0.78      0.74     25116
weighted avg       0.82      0.78      0.79     25116



In [24]:
df_importance=pd.DataFrame({"tag":Xtrain.columns,"importance":pipe["clf"].feature_importances_}).sort_values(by="importance",ascending=False).reset_index(drop=True)
df_importance.head()

,tag,importance
0,min_solid percentage_5,0.120459
1,solid percentage,0.096574
2,min_solid percentage_3,0.088662
3,rms_delta LL charge cell_3,0.024193
4,min_delta LL charge cell_10,0.017377


In [25]:
pd.options.plotting.backend = "plotly"
fig=df_importance["importance"].plot(kind='hist')
fig.update_layout(height=500, width=1200,title="Feature importance XGBoost (Gain)",) 
fig.show()

In [26]:
# Plot
fig = go.Figure([go.Bar(x=df_importance["tag"], y=df_importance["importance"])])
fig.update_layout(height=500, width=1200,title="Feature importance XGBoost (Gain)",) 
fig.show()

In [27]:
xgboost_fs=df_importance[:100].tag.to_list()
xgboost_fs

['min_solid percentage_5',
 'solid percentage',
 'min_solid percentage_3',
 'rms_delta LL charge cell_3',
 'min_delta LL charge cell_10',
 'var_delta LL charge cell_3',
 'var_delta HH charge cell_10',
 'var_delta HH charge cell_3',
 'min_speed_10',
 'var_delta LL charge cell_10',
 'mean_LL charge cell_10',
 'rms_crusher index_5',
 'max_delta LL charge cell_10',
 'min_solid percentage_10',
 'max_LL charge cell_5',
 'max_covelin law_5',
 'LL charge cell',
 'ball work index_(t-1)',
 'mean_chalcocite law_5',
 'min_pyrite law_3',
 'var_solid percentage_10',
 'rms_speed_5',
 'rms_delta LL charge cell_10',
 'HH charge cell_(t-2)',
 'rms_sag power index_10',
 'var_solid percentage_5',
 'min_speed_3',
 'rms_chalcocite law_5',
 'var_charge cell_10',
 'delta LL charge cell',
 'HH charge cell_(t-5)',
 'chalcopyrite law_(t-2)',
 'rms_LL charge cell_10',
 'mean_pyrite law_5',
 'rms_HH charge cell_10',
 'min_delta HH charge cell_10',
 'speed',
 'HH charge cell_(t-3)',
 'min_bornite law_5',
 'min_sag 

In [28]:
# Se obtienen tags altamente correlacionados y matriz que permite visualizar cuales son los par de tags altamente correlacionados
collinearity, table_cor=corrkill(Xtrain,xgboost_fs,corr_cut=0.9)

In [29]:
# Tags altamente correlacionados
table_cor

,Feat1,Feat2,Val
3483,rms_HH charge cell_10,mean_HH charge cell_10,0.999999
1032,mean_LL charge cell_10,rms_LL charge cell_10,0.999999
1083,mean_LL charge cell_10,mean_HH charge cell_10,0.999996
3234,rms_LL charge cell_10,rms_HH charge cell_10,0.999996
1694,LL charge cell,HH charge cell,0.999995
...,...,...,...
3572,min_delta HH charge cell_10,delta HH charge cell,0.910151
1229,max_delta LL charge cell_10,delta LL charge cell,0.910140
1272,max_delta LL charge cell_10,delta HH charge cell,0.910137
2935,delta LL charge cell,min_delta HH charge cell_10,0.910128


In [30]:
tag_select = [i for i in xgboost_fs if i not in collinearity]
tag_select

['min_solid percentage_5',
 'rms_delta LL charge cell_3',
 'min_delta LL charge cell_10',
 'var_delta LL charge cell_3',
 'var_delta HH charge cell_10',
 'min_speed_10',
 'mean_LL charge cell_10',
 'rms_crusher index_5',
 'max_delta LL charge cell_10',
 'max_covelin law_5',
 'ball work index_(t-1)',
 'mean_chalcocite law_5',
 'min_pyrite law_3',
 'var_solid percentage_10',
 'rms_sag power index_10',
 'var_solid percentage_5',
 'chalcopyrite law_(t-2)',
 'min_bornite law_5',
 'HH TPH',
 'var_LL charge cell_10',
 'Edad',
 'var_speed_10',
 'var_covelin law_3',
 'charge cell_(t-3)']

In [31]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBClassifier(seed=1,  eval_metric='mlogloss'))
    ]
)

In [32]:
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['min_solid percentage_5',
                                                   'rms_delta LL charge cell_3',
                                                   'min_delta LL charge '
                                                   'cell_10',
                                                   'var_delta LL charge cell_3',
                                                   'var_delta HH charge '
                                                   'cell_10',
                                                   'min_speed_10',
                                                   'mean_LL charge cell_10',
                                                   'rms_crusher index_5',
                                                   'max_delta LL charge '
                                                   'cell_10',
                      

**General**

In [33]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest)
print(classification_report(Ytest,Ypred))

              precision    recall  f1-score   support

         0.0       0.92      0.82      0.87     94376
         1.0       0.65      0.83      0.73     39276

    accuracy                           0.82    133652
   macro avg       0.79      0.82      0.80    133652
weighted avg       0.84      0.82      0.83    133652



# GridSearch

In [34]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        #features.columns.to_list()
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
        #("Selection", SelectPercentile(f_classif, percentile=10)),
        #("PCA",PCA(n_components=150 )), #0.08
     ("reg",  xgb.XGBClassifier(seed=1,  eval_metric='mlogloss'))
    ]
)


param_grid_reg = {'reg__learning_rate': list(np.linspace(0.001,1,10)), # Boosting learning rate
                  'reg__gamma':list(np.linspace(0.0001,50,5)), # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                  #'reg__max_depth':[5,10,12,15,20], # Maximum tree depth for base learners.
                  #'reg__min_child_weight':[10,5,1,2,3], # Minimum sum of instance weight(hessian) needed in a child.
                  #'reg__colsample_bytree':[0.5,0.8,1], # Subsample ratio of columns when constructing each tree.
                  'reg__n_estimators': [100,200,300], # Number of gradient boosted trees.   
                  #'reg__reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              #'reg__reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
               #'Selection__percentile': [10,20,30,50,70,80,100]              
                 } 

# Clasificación
gs_reg = HalvingGridSearchCV(pipe, param_grid_reg, scoring='neg_mean_absolute_error',cv=x,refit=True,
                             verbose=10,
                             aggressive_elimination=True
                            )

# Fit
gs_reg.fit(Entre_Vali_features, Entre_Vali_target)


n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 12282
max_resources_: 994906
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 150
n_resources: 12282
Fitting 1 folds for each of 150 candidates, totalling 150 fits
[CV 1/1; 1/150] START reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=100
[CV 1/1; 1/150] END reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=100;, score=(train=-0.073, test=-0.189) total time=   9.1s
[CV 1/1; 2/150] START reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=200
[CV 1/1; 2/150] END reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=200;, score=(train=-0.066, test=-0.184) total time=   0.5s
[CV 1/1; 3/150] START reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=300
[CV 1/1; 3/150] END reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=300;, score=(train=-0.063, test=-0.180) total time=   0.7s
[CV 1/1; 4/150] START reg__gamma=0.000

HalvingGridSearchCV(aggressive_elimination=True,
                    cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                    estimator=Pipeline(steps=[('preprocesamiento',
                                               ColumnTransformer(transformers=[('MinMax',
                                                                                MinMaxScaler(),
                                                                                ['min_solid '
                                                                                 'percentage_5',
                                                                                 'rms_delta '
                                                                                 'LL '
                                                                                 'charge '
                                                                                 'cell_3',
                                                                

In [36]:
gs_reg.best_params_

{'reg__gamma': 50.0, 'reg__learning_rate': 0.112, 'reg__n_estimators': 300}

In [37]:
# Métricas de evaluación 
Ypred= gs_reg.predict(Xtest)
print(classification_report(Ytest,Ypred))

              precision    recall  f1-score   support

         0.0       0.93      0.81      0.87     94376
         1.0       0.65      0.84      0.74     39276

    accuracy                           0.82    133652
   macro avg       0.79      0.83      0.80    133652
weighted avg       0.85      0.82      0.83    133652



# Train model

In [38]:
tag_select

['min_solid percentage_5',
 'rms_delta LL charge cell_3',
 'min_delta LL charge cell_10',
 'var_delta LL charge cell_3',
 'var_delta HH charge cell_10',
 'min_speed_10',
 'mean_LL charge cell_10',
 'rms_crusher index_5',
 'max_delta LL charge cell_10',
 'max_covelin law_5',
 'ball work index_(t-1)',
 'mean_chalcocite law_5',
 'min_pyrite law_3',
 'var_solid percentage_10',
 'rms_sag power index_10',
 'var_solid percentage_5',
 'chalcopyrite law_(t-2)',
 'min_bornite law_5',
 'HH TPH',
 'var_LL charge cell_10',
 'Edad',
 'var_speed_10',
 'var_covelin law_3',
 'charge cell_(t-3)']

In [17]:
tag_select=['min_solid percentage_5',
 'rms_delta LL charge cell_3',
 'min_delta LL charge cell_10',
 'var_delta LL charge cell_3',
 'var_delta HH charge cell_10',
 'min_speed_10',
 'mean_LL charge cell_10',
 'rms_crusher index_5',
 'max_delta LL charge cell_10',
 'max_covelin law_5',
 'ball work index_(t-1)',
 'mean_chalcocite law_5',
 'min_pyrite law_3',
 'var_solid percentage_10',
 'rms_sag power index_10',
 'var_solid percentage_5',
 'chalcopyrite law_(t-2)',
 'min_bornite law_5',
 'HH TPH',
 'var_LL charge cell_10',
 'Edad',
 'var_speed_10',
 'var_covelin law_3',
 'charge cell_(t-3)']

In [39]:
gs_reg.best_params_

{'reg__gamma': 50.0, 'reg__learning_rate': 0.112, 'reg__n_estimators': 300}

In [18]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf",   xgb.XGBClassifier(seed=1,  eval_metric='mlogloss',gamma= 50, learning_rate= 0.112, n_estimators= 300))
    ]
)
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['min_solid percentage_5',
                                                   'rms_delta LL charge cell_3',
                                                   'min_delta LL charge '
                                                   'cell_10',
                                                   'var_delta LL charge cell_3',
                                                   'var_delta HH charge '
                                                   'cell_10',
                                                   'min_speed_10',
                                                   'mean_LL charge cell_10',
                                                   'rms_crusher index_5',
                                                   'max_delta LL charge '
                                                   'cell_10',
                      

**General**

In [20]:
Ytest.value_counts()

0.0    94376
1.0    39276
Name: loss of TPH, dtype: int64

In [22]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest)
print(classification_report(Ytest,Ypred))

              precision    recall  f1-score   support

         0.0       0.93      0.80      0.86     94376
         1.0       0.65      0.86      0.74     39276

    accuracy                           0.82    133652
   macro avg       0.79      0.83      0.80    133652
weighted avg       0.85      0.82      0.83    133652



In [24]:
# Matriz de confusión
matrix=confusion_matrix(Ytest,Ypred,normalize='true'
)
z=[matrix[0].round(2),matrix[1].round(2)]
fig = px.imshow(z, aspect="auto",labels=dict(x="Predicted label", y="True label"),x=["0", "1"],y=["0", "1"]
)

#axis_template = dict(showticklabels = False,ticks = '')
fig.update_layout(height=600,width=600,#title="Matriz de confusión normalizada",
# xaxis = axis_template,yaxis = axis_template,
)
fig.update_xaxes(side="top")
for i,r in enumerate(matrix):
    for k,c in enumerate(r):
        fig.add_annotation(x=k,y=i,
                           text='{}'.format(round(c,2)),
                           showarrow=False, bgcolor="white"
    )
                          
fig.show()

In [23]:
# Matriz de confusión
matrix=confusion_matrix(Ytest,Ypred,#normalize='true'
)
z=[matrix[0].round(2),matrix[1].round(2)]
fig = px.imshow(z, aspect="auto",labels=dict(x="Predicted label", y="True label"),x=["0", "1"],y=["0", "1"]
)

#axis_template = dict(showticklabels = False,ticks = '')
fig.update_layout(height=600,width=600,#title="Matriz de confusión normalizada",
# xaxis = axis_template,yaxis = axis_template,
)
fig.update_xaxes(side="top")
for i,r in enumerate(matrix):
    for k,c in enumerate(r):
        fig.add_annotation(x=k,y=i,
                           text='{}'.format(round(c,2)),
                           showarrow=False, bgcolor="white"
    )
                          
fig.show()